In [186]:
import numpy as np
import pandas as pd
import requests
import yfinance as yf
import prophet
from prophet import Prophet
from prophet.diagnostics import cross_validation, performance_metrics
import seaborn as sns
import matplotlib.pyplot as plt
import prophet.plot as ppl
import datetime as dt
import itertools

### Start Date and Today

In [187]:
from datetime import date

start_date = "2017-01-01"
today = date.today().strftime("%Y-%m-%d")

### Load the Data

In [188]:
ticker = ["GOOG"]
data = yf.download(ticker, start_date, today)

[*********************100%***********************]  1 of 1 completed


### Data Manipulation of Stock Data

In [189]:
data.reset_index(inplace=True)
data["Date"] = data["Date"].dt.tz_localize(None)

In [190]:
data.head(), data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1528 entries, 0 to 1527
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Date       1528 non-null   datetime64[ns]
 1   Open       1528 non-null   float64       
 2   High       1528 non-null   float64       
 3   Low        1528 non-null   float64       
 4   Close      1528 non-null   float64       
 5   Adj Close  1528 non-null   float64       
 6   Volume     1528 non-null   int64         
dtypes: datetime64[ns](1), float64(5), int64(1)
memory usage: 83.7 KB


(        Date       Open       High        Low      Close  Adj Close     Volume
 0 2017-01-03  28.950001  29.082500  28.690001  29.037500  27.174759  115127600
 1 2017-01-04  28.962500  29.127501  28.937500  29.004999  27.144344   84472400
 2 2017-01-05  28.980000  29.215000  28.952499  29.152500  27.282383   88774400
 3 2017-01-06  29.195000  29.540001  29.117500  29.477501  27.586529  127007600
 4 2017-01-09  29.487499  29.857500  29.485001  29.747499  27.839209  134247600,
 None)

In [191]:
df_train = data[["Date", "Close"]]
df_train = df_train.rename(columns={"Date": "ds", "Close": "y"})

In [192]:
df_train

,ds,y
0,2017-01-03,29.037500
1,2017-01-04,29.004999
2,2017-01-05,29.152500
3,2017-01-06,29.477501
4,2017-01-09,29.747499
...,...,...
1523,2023-01-23,141.110001
1524,2023-01-24,142.529999
1525,2023-01-25,141.860001
1526,2023-01-26,143.960007


### Model

In [193]:
baseline_model = Prophet()
baseline_model.fit(df_train)

03:13:07 - cmdstanpy - INFO - Chain [1] start processing
03:13:07 - cmdstanpy - INFO - Chain [1] done processing


### Hyperparameter Tuning

In [ ]:
# Some code here

### Save Model

In [194]:
# from prophet.serialize import model_to_json, model_from_json
#
# with open('serialized_model.json', 'w') as fout:
#     fout.write(model_to_json(auto_model))  # Save model
#
# with open('serialized_model.json', 'r') as fin:
#     auto_model = model_from_json(fin.read())  # Load model

### Predict Future

In [195]:
period = 365 * 2
preds = baseline_model.predict(df_train)
future = baseline_model.make_future_dataframe(periods=period)
forecast = baseline_model.predict(future)

In [196]:
forecast.tail()

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,weekly,weekly_lower,weekly_upper,yearly,yearly_lower,yearly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
2253,2025-01-22,107.367923,46.301058,173.128388,46.935542,168.394480,2.506277,2.506277,2.506277,0.558740,0.558740,0.558740,1.947537,1.947537,1.947537,0.0,0.0,0.0,109.874200
2254,2025-01-23,107.320274,46.055316,171.334820,46.885717,168.487007,2.506851,2.506851,2.506851,0.388110,0.388110,0.388110,2.118741,2.118741,2.118741,0.0,0.0,0.0,109.827125
2255,2025-01-24,107.272625,47.053656,172.705303,46.835891,168.598755,2.569408,2.569408,2.569408,0.268655,0.268655,0.268655,2.300753,2.300753,2.300753,0.0,0.0,0.0,109.842034
2256,2025-01-25,107.224976,46.181675,170.995116,46.786065,168.710503,1.344168,1.344168,1.344168,-1.147399,-1.147399,-1.147399,2.491567,2.491567,2.491567,0.0,0.0,0.0,108.569144
2257,2025-01-26,107.177328,44.341262,174.010507,46.713760,168.822251,1.541476,1.541476,1.541476,-1.147400,-1.147400,-1.147400,2.688876,2.688876,2.688876,0.0,0.0,0.0,108.718804


### Plot

In [197]:
ppl.plot_plotly(baseline_model, forecast.loc[forecast["yhat"] > 0])